In [1]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
import numpy as np
import tensorflow as tf

In [2]:
with open("all_data.pkl", "rb") as f:
    all_data = pickle.load(f)

In [3]:
def build_labels(data):
    labels = np.zeros([len(data), 9])
    label2idx = {}
    idx2label = {}
    idx = 0
    for item in data:
        if item[0] not in label2idx.keys():
            label2idx[item[0]] = idx
            idx2label[idx] = item[0]
            idx += 1
    for (i, item) in enumerate(data):
        labels[i][label2idx[item[0]]] = 1
    return labels, label2idx, idx2label

In [4]:
labels, label2idx, idx2label = build_labels(all_data)

In [ ]:
labels[0]

In [5]:
corpus = []
for item in all_data:
    corpus.append(' '.join(item[1]))

In [6]:
tokenizer=Tokenizer(15000)
tokenizer.fit_on_texts(corpus)
vocab=tokenizer.word_index
x_train, x_test, y_train, y_test = train_test_split(corpus, labels, test_size=0.1)
x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)
x_train = pad_sequences(x_train, maxlen=100)
x_test = pad_sequences(x_test, maxlen=100)

In [10]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(15000, 64),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(9, activation='softmax')
])
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [11]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 64)          960000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 9)                 585       
Total params: 997,769
Trainable params: 997,769
Non-trainable params: 0
_________________________________________________________________


In [12]:
history = model.fit(x_train, y_train, epochs=100, batch_size=512, verbose=1, validation_split = 0.15)

Train on 10425 samples, validate on 1840 samples
Epoch 1/100
10425/10425 [==============================] - 13s 1ms/sample - loss: 2.1936 - accuracy: 0.2173 - val_loss: 2.1893 - val_accuracy: 0.2582
Epoch 2/100
10425/10425 [==============================] - 9s 895us/sample - loss: 2.1847 - accuracy: 0.2576 - val_loss: 2.1780 - val_accuracy: 0.2467
Epoch 3/100
10425/10425 [==============================] - 15s 1ms/sample - loss: 2.1683 - accuracy: 0.2454 - val_loss: 2.1518 - val_accuracy: 0.2413
Epoch 4/100
10425/10425 [==============================] - 12s 1ms/sample - loss: 2.1109 - accuracy: 0.2433 - val_loss: 2.0249 - val_accuracy: 0.2413
Epoch 5/100
10425/10425 [==============================] - 7s 677us/sample - loss: 2.0170 - accuracy: 0.2433 - val_loss: 1.9923 - val_accuracy: 0.2413
Epoch 6/100
10425/10425 [==============================] - 8s 741us/sample - loss: 1.9820 - accuracy: 0.2433 - val_loss: 1.9550 - val_accuracy: 0.2413
Epoch 7/100
10425/10425 [=======================

KeyboardInterrupt: 

In [21]:
model.evaluate(x_test, y_test)

1363/1363 [==============================] - 0s 79us/sample - loss: 0.5671 - accuracy: 0.8225


[0.5670529038523019, 0.82245046]

In [16]:
x_train.shape

(12265, 100)